In [8]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import re  # For handling space-aligned labels correctly

# def getModtranData(run):
#     with open(run + '/tape7.scn') as fp:
#         step1 = fp.readlines()
#         activate = False
#         data = []
        
#         for line in step1:
#             if "WAVLEN MCRN" in line:  # Ensure full label match
#                 activate = True
          
#                 labels = re.findall(r'\S+(?: \S+)?', line.strip())
#                 labels = [label for label in labels if label not in ['THRML SCT', 'DEPTH']]
                
#             elif activate:
#                 if line.strip() == '-9999.':
#                     activate = False
#                     continue
#                 else:
#                     data.append(line.strip().split())

#         # Convert list to numpy array (handle empty case)
#         if data:
#             data2 = np.array(data, dtype=np.float32)
#             runData = {label: data2[:, i] for i, label in enumerate(labels)}
#         else:
#             runData = {}

#     return runData

# # Usage 
# runData = getModtranData('MODTRAL_models_2025/MLS_ALB0_WAT0.2')


In [7]:
# import numpy as np
# import os
# import re

# def getModtranData(run):
#     file_path = os.path.join(run, 'tape7.scn')
#     with open(file_path) as fp:
#         step1 = fp.readlines()
#         activate = False
#         data = []
#         labels = []
        
#         for line in step1:
#             if "WAVLEN" in line:
#                 activate = True
#                 labels = re.split(r'\s{2,}', line.strip())  # Split on 2+ spaces
                
#                 # Remove unwanted labels safely
#                 for unwanted in ['THRML SCT', 'DEPTH']:
#                     if unwanted in labels:
#                         labels.remove(unwanted)
                
#             elif activate:
#                 if line.strip() == '-9999.':
#                     activate = False
#                     continue
#                 else:
#                     data.append(line.strip().split())

#         data2 = np.float32(data)
#         runData = {label: data2[:, i] for i, label in enumerate(labels)}
        
#     return runData

# # base_dir = os.getcwd()  
# # modtran_path = os.path.join(base_dir, 'MODTRAL_models_2025', 'MLS_ALB0_WAT0.2')
# runData = getModtranData('MODTRAL_models_2025/MLS_ALB0_WAT0.2')

In [ ]:
# def tape7scn(file_path):
#     data = {'WAVLEN MCRN': [], 'GRND RFLT': [], 'TOTAL RAD': []}
    
#     with open(file_path, 'r') as f:
#         lines = f.readlines()
        
#         for line in lines[11:]:  # Start from line 12 (index 11)
#             wavlen_mcrn = line[4:13].strip()  # Columns 5-13
#             grnd_rflt = line[75:86].strip()   # Columns 76-86
#             total_rad = line[97:108].strip()  # Columns 98-108
            
#             if wavlen_mcrn and grnd_rflt and total_rad:
#                 data['WAVLEN MCRN'].append(wavlen_mcrn)
#                 data['GRND RFLT'].append(grnd_rflt)
#                 data['TOTAL RAD'].append(total_rad)
    
#     return data

In [3]:
import numpy as np
import os
import re
import pandas as pd
from pathlib import Path

def getModtranData(run):
    file_path = os.path.join(run, 'tape7.scn')
    with open(file_path) as fp:
        step1 = fp.readlines()
        activate = False
        data = []
        labels = []
        
        for line in step1:
            if "WAVLEN" in line:
                activate = True
                labels = re.split(r'\s{2,}', line.strip()) # Handle multi-word entries
                
                for unwanted in ['THRML SCT', 'DEPTH']: # Remove unwanted labels safely
                    if unwanted in labels:
                        labels.remove(unwanted)
                
            elif activate:
                if line.strip() == '-9999.':
                    activate = False
                    continue
                else:
                    data.append(line.strip().split())

        data2 = np.float32(data)
        runData = {label: data2[:, i] for i, label in enumerate(labels)}
        
    return runData

def process_all_profiles(main_dir):
    # Dictionary to hold DataFrames keyed by profile name
    profile_dfs = {}
    
    # Group directories by profile and water vapor
    profile_groups = {}
    
    # Walk through directory tree
    for root, dirs, files in os.walk(main_dir):
        for dir_name in dirs:
            # Match pattern: PROFILE_ALBx_WATy.y (e.g., "MLS_ALB0_WAT0.2")
            match = re.match(r"^([A-Za-z]+)_ALB(\d+)_WAT([\d.]+)$", dir_name)
            if match:
                profile, alb, wvp = match.groups()
                wvp = float(wvp)  # Convert to numeric
                
                # Create grouping key
                key = (profile, wvp)
                
                # Initialize group if new
                if key not in profile_groups:
                    profile_groups[key] = {'ALB0': None, 'ALB1': None}
                
                # Store path by albedo type
                run_path = os.path.join(root, dir_name)
                profile_groups[key][f'ALB{alb}'] = run_path

    # Process each profile group
    for (profile, wvp), paths in profile_groups.items():
        
        missing = []
        if not paths['ALB0']:
            missing.append(f"{profile}_ALB0_WAT{wvp:.1f}")
        if not paths['ALB1']:
            missing.append(f"{profile}_ALB1_WAT{wvp:.1f}")
        
        if missing:
            error_msg = f"Skipping incomplete pair: Missing {', '.join(missing)}"
            if paths['ALB0'] or paths['ALB1']:
                existing = [p for p in [paths['ALB0'], paths['ALB1']] if p]
                error_msg += f" (Only found: {', '.join(os.path.basename(p) for p in existing)})"
            print(error_msg)
            continue            
        try:
            # Load ALB0 data (TOTAL_RAD)
            alb0_data = getModtranData(paths['ALB0'])
            alb0_df = pd.DataFrame({
                'WAVLEN_MCRN': alb0_data['WAVLEN MCRN'],
                'TOTAL_RAD': alb0_data['TOTAL RAD']
            })
            
            # Load ALB1 data (GRND_RFLCT)
            alb1_data = getModtranData(paths['ALB1'])
            alb1_df = pd.DataFrame({
                'WAVLEN_MCRN': alb1_data['WAVLEN MCRN'],
                'GRND_RFLCT': alb1_data['GRND RFLT']
            })
            
            # Merge on wavelength
            merged = pd.merge(alb0_df, alb1_df, on='WAVLEN_MCRN', how='inner')
            
            # Add metadata
            merged['PROFILE'] = profile
            merged['WATER'] = wvp
            
            # Add to final DataFrames
            if profile not in profile_dfs:
                profile_dfs[profile] = merged
            else:
                profile_dfs[profile] = pd.concat([profile_dfs[profile], merged])
                
        except Exception as e:
            dir_names = [os.path.basename(p) for p in paths.values()]
            print(f"Error processing {profile} pair: {', '.join(dir_names)}")
            print(f"Error details: {str(e)}")
    
    return profile_dfs

# Example usage
main_dir = 'MODTRAL_models_2025'
profile_dataframes = process_all_profiles(main_dir)

# Report here
if not profile_dataframes:
    print("No valid atmospheric profiles found!")
else:
    # Process all profiles
    for profile_name, df in profile_dataframes.items():
        # 1. Save to CSV
        filename = f"{profile_name}_atmospheric_profile.csv"
        df.to_csv(filename, index=False)
        
        # 2. Access DataFrame for analysis
        print(f"\nProfile: {profile_name}")
        print(f"Data Shape: {df.shape}")
        print(df)
        

Error processing SAW pair: SAW_ALB0_WAT0.5, SAW_ALB1_WAT0.5
Error details: too many indices for array: array is 1-dimensional, but 2 were indexed
Error processing SAW pair: SAW_ALB0_WAT1.0, SAW_ALB1_WAT1.0
Error details: too many indices for array: array is 1-dimensional, but 2 were indexed

Profile: MLS
Data Shape: (11255, 5)
      WAVLEN_MCRN     TOTAL_RAD  GRND_RFLCT PROFILE  WATER
0           0.350  6.004700e-03    0.009890     MLS    0.2
1           0.351  5.866200e-03    0.009821     MLS    0.2
2           0.352  6.039800e-03    0.010291     MLS    0.2
3           0.353  5.603100e-03    0.009723     MLS    0.2
4           0.354  6.348600e-03    0.011199     MLS    0.2
...           ...           ...         ...     ...    ...
2246        2.596  3.719000e-08    0.000000     MLS    2.0
2247        2.597  4.941700e-08    0.000000     MLS    2.0
2248        2.598  5.588200e-08    0.000000     MLS    2.0
2249        2.599  7.014900e-08    0.000000     MLS    2.0
2250        2.600  6.3

In [ ]:
import sys

def createTape5(atmospheric_model, water, albedo, scat):
    # Validate scat value
    if scat not in [-1, 1, 0]:
        raise ValueError("scat must be either -1 or 1")

    atm_model = int(atmospheric_model)
    
    # Ensure strict decimal places
    water_vapor = f"{water:.3f}"  # Always 3 decimal places
    albedo_str = f"{albedo:.2f}"  # Always 2 decimal places
    scat_val = f"{scat:>5}" # Right-align scat in a 5 charater field

    with open('tape5', 'w') as the_file:
        the_file.write(f'TS  {atm_model}    2    2{scat_val}    0    0    0    0    0    0    1    0    0 283.150   {albedo_str}\n')
        the_file.write(f'T   8T   0 360.00000     {water_vapor}         0 T F F         0.000\n')
        the_file.write('\n')
        the_file.write('    1    1    0    0    0    0     0.000     0.000     0.000     0.000     1.000\n')
        the_file.write(' 705.00000     1.000   180.000     0.000     0.000     0.000    0          0.000\n')
        the_file.write('   1     2  241    0\n')
        the_file.write('    43.680    77.490     0.000     0.000    16.000     0.000     0.000     0.000\n')
        the_file.write('     0.350     2.600     0.001     0.001RM        MR A\n')
        the_file.write('    0\n')

if __name__ == "__main__":
    if len(sys.argv) != 5:  # Updated to accept 4 arguments
        print("Usage: python modify_tape5.py <atmospheric_model> <water> <albedo> <scat>")
        sys.exit(1)

    atm = int(sys.argv[1])
    water = float(sys.argv[2])
    albedo = float(sys.argv[3])
    scatter = int(sys.argv[4])  
    try:
        createTape5(atm, water, albedo, scatter)
        print("✅ Modified `tape5` file successfully created!")
    except ValueError as e:
        print(f"❌ Error: {e}")
        sys.exit(1)

In [ ]:
# Process tape7.scn
def getModtranData(run):
    file_path = os.path.join(run, 'tape7.scn')
    with open(file_path) as fp:
        step1 = fp.readlines()
        activate = False
        data = []
        labels = []
        
        for line in step1:
            if "WAVLEN" in line:
                activate = True
                labels = re.split(r'\s{2,}', line.strip()) # Handle multi-word entries
                
                for unwanted in ['THRML SCT', 'DEPTH']: # Remove unwanted labels safely
                    if unwanted in labels:
                        labels.remove(unwanted)
                
            elif activate:
                if line.strip() == '-9999.':
                    activate = False
                    continue
                else:
                    data.append(line.strip().split())

        data2 = np.float32(data)
        runData = {label: data2[:, i] for i, label in enumerate(labels)}
        
    return runData

In [ ]:
import os

def readTape6(run1):
    water_profile_info = {}
    tape6_path = os.path.join(run1, 'subfolder', 'tape6')  # Adjust 'subfolder' as needed

    try:
        if os.path.exists(tape6_path):
            with open(tape6_path, 'r') as fp:
                lines = fp.readlines()
                for i, line in enumerate(lines):
                    if "THE WATER PROFILE WAS DECREASED TO FIT THE INPUT WATER COLUMN VALUE" in line:
                        
                        initial_line = lines[i + 1].strip()
                        input_line = lines[i + 2].strip()
                        final_line = lines[i + 3].strip()

                        initial_value = initial_line.split(':')[1].strip()
                        input_value = input_line.split(':')[1].strip()
                        final_value = final_line.split(':')[1].strip()

                        water_profile_info = {'initial': initial_value, 'input': input_value, 'final': final_value}
                        return water_profile_info  
        else:
            print(f"tape6 file not found at: {tape6_path}")
    except FileNotFoundError:
        print(f"File not found: {tape6_path}")
    except Exception as e:
        print(f"Error reading tape6 file: {e}")
    
    return water_profile_info

In [1]:
# # Main
# if __name__ == "__main__":
#     main_dir = 'MODTRAN_models_2025_b'
#     profile_dataframes, folders_accessed = process_tape7_scn(main_dir)
#     tape6Data = readTape6(main_dir, folders_accessed)

#     ATM_DATA_FRAME = {}
#     for profile, df in profile_dataframes.items():
#         df["INITIAL_H2O"] = df["INITIAL_H2O"].astype(float)
#         tape6Data["INITIAL_H2O"] = tape6Data["INITIAL_H2O"].astype(float)

#     for profile, df in profile_dataframes.items():
#         df["FINAL_H2O"] = np.nan  
        
#         for i, row in tape6Data.iterrows():
#             matching_row = df[df["INITIAL_H2O"] == row["INITIAL_H2O"]]
#             if not matching_row.empty and row["ATM_PROFILE"] == profile:
#                 final_h2o_value = float(row["FINAL_H2O"])  # Convert to float
#                 df.loc[df["INITIAL_H2O"] == row["INITIAL_H2O"], "FINAL_H2O"] = final_h2o_value
        
#         ATM_DATA_FRAME[profile] = df 


In [ ]:
# spectra_dict = {}
# for filename in ECOSTRESS_path():
#     file_path = os.path.join('ecostress_download', filename)
#     oak_wav_micron, oak_reflec = read_ECOSTRESS_data(file_path)
#     modtran_wavelen = DATA_FRAME['MLW']['WAVLEN_MCRN']
#     reflect_downsmpl = downsample(oak_wav_micron, oak_reflec, modtran_wavelen)

#      # Sort the data by wavelength 
#     sample_name = filename.split('.')[0]
#     sorted_pairs = sorted(zip(modtran_wavelen, reflect_downsmpl))
#     sample_wave, sample_reflect = zip(*sorted_pairs)
#     spectra_dict[sample_name] = sample_wave, sample_reflect


# wave = DATA_FRAME['MLW']['WAVLEN_MCRN']
# oak_wav_micron, oak_reflec = read_ECOSTRESS_data(file)
# asphalt_wav_micron, asphalt_reflec = read_ECOSTRESS_data(file)
# oak_downsmpl = downsample(oak_wav_micron, oak_reflec, wave)
# asphalt_downsmpl = downsample(asphalt_wav_micron, asphalt_reflec, wave)

# # Sort the data by wavelength
# sorted_pairs = sorted(zip(modtran_wavelen, oak_downsmpl))
# wave_oak, oak_reflect = zip(*sorted_pairs)

# sorted_pairs = sorted(zip(wave, asphalt_downsmpl))
# wave_asphalt, asphalt_reflect = zip(*sorted_pairs)


In [ ]:
# # Create a figure and subplot
# fig, ax = plt.subplots(1, 2, figsize=(8, 6))
# # Plot Oak data on the first subplot
# ax[0].plot(wave_oak, oak_reflect, label='Oak', color='green', linestyle='-', linewidth=2)
# ax[0].set_xlabel('Wavelength (microns)', fontsize=12)
# ax[0].set_ylabel('Reflectance', fontsize=12)
# ax[0].set_title('Oak Reflectance', fontsize=14)
# ax[0].legend(loc='upper right', fontsize=10)
# ax[0].grid(True, linestyle='--', alpha=0.6)

# # Plot Asphalt data on the second subplot
# ax[1].plot(wave_asphalt, asphalt_reflect, label='Asphalt', color='gray', linestyle='-', linewidth=2)
# ax[1].set_xlabel('Wavelength (microns)', fontsize=12)
# ax[1].set_ylabel('Reflectance', fontsize=12)
# ax[1].set_title('Asphalt Reflectance', fontsize=14)
# ax[1].legend(loc='upper right', fontsize=10)
# ax[1].grid(True, linestyle='--', alpha=0.6)

# plt.tight_layout()  # Adjust layout to prevent overlap
# plt.show()



In [ ]:
# # LANDIS RSR Analysis
# import numpy as np
# import os
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker

# # repository path
# output_dir = "RSR_functions"
# os.makedirs(output_dir, exist_ok=True)

# # Landsat Next Bands (CWV in microns)
# bands = {
#     "Violet": 0.412, "Coastal_Aerosol": 0.442, "Blue": 0.490, "Green": 0.560, "Yellow": 0.600,
#     "Orange": 0.620, "Red_1": 0.650, "Red_2": 0.665, "Red_Edge_1": 0.705, "Red_Edge_2": 0.740,
#     "NIR_Broad": 0.865, "NIR1": 0.945, "Water_Vapor": 1.035, "SWIR1": 1.375,
#     "SWIR2a": 1.610, "SWIR2b": 2.105, "TIR_1": 3.100, "TIR_2": 8.600, 
#     "TIR_3": 9.100, "TIR_4": 11.300, "TIR_5": 12.000}

# # threshold for band limits (1% of peak)
# THRESHOLD = 0.01

# # Generate RSR data
# sigma_values = {band: cwv * 0.05 for band, cwv in bands.items()}
# wavelengths = np.linspace(0.3, 13.0, 10000)

# rsr_data = {}
# band_limits = {}  
# for band, cwv in bands.items():
#     sigma = sigma_values[band]
#     rsr = np.exp(-((wavelengths - cwv) ** 2) / (2 * sigma ** 2))
    
#     # Compute band limits using threshold
#     delta = sigma * np.sqrt(-2 * np.log(THRESHOLD))
#     left_limit = cwv - delta
#     right_limit = cwv + delta
#     band_limits[band] = (left_limit, right_limit)
    
#     # Save data
#     file_path = os.path.join(output_dir, f"{band}_RSR.txt")
#     with open(file_path, "w") as f:
#         f.write("Wavelength\tRSR\n")
#         for wl, rsr_val in zip(wavelengths, rsr):
#             f.write(f"{wl:.4f}\t{rsr_val:.6f}\n")
    
#     df = pd.DataFrame({"Wavelength": wavelengths, "RSR": rsr})
#     rsr_data[band] = df

# # Create subplots for Band 12, 13, and 15
# fig, ax = plt.subplots(1, 1, figsize=(15, 5))

# def get_trimmed_data(band_name):
#     left, right = band_limits[band_name]
#     df = rsr_data[band_name]
#     mask = (df["Wavelength"] >= left) & (df["Wavelength"] <= right)
#     return df["Wavelength"][mask], df["RSR"][mask]

# # Get trimmed data for plotting
# nir1_wave, nir1_rsr = get_trimmed_data("NIR1")
# water_vapor_wave, water_vapor_rsr = get_trimmed_data("Water_Vapor")
# swir2a_wave, swir2_rsrs = get_trimmed_data("SWIR2a")

# # Dictionary of DataFrames (Band limits)
# SENSOR_RSR_FRAME = {
#     "NIR1": pd.DataFrame({"Wavelength": nir1_wave, "RSR": nir1_rsr }),
#     "Water_Vapor": pd.DataFrame({"Wavelength": water_vapor_wave, "RSR": water_vapor_rsr}),
#     "SWIR2a": pd.DataFrame({"Wavelength": swir2a_wave, "RSR": swir2_rsrs})}

# # Plot
# ax.plot(nir1_wave, nir1_rsr, label="NIR1 (Band12)", linewidth=2)
# ax.plot(water_vapor_wave, water_vapor_rsr, label="Water Vapor (Band13)", linewidth=2)
# ax.plot(swir2a_wave, swir2_rsrs, label="SWIR2a (Band15)", linewidth=2)

# # Add vertical lines to mark band edges
# for band_name, color in zip(["NIR1", "Water_Vapor", "SWIR2a"], ["blue", "green", "red"]):
#     left, right = band_limits[band_name]
#     ax.axvline(left, color=color, linestyle="--", alpha=0.5)
#     ax.axvline(right, color=color, linestyle="--", alpha=0.5)

# ax.set_xlabel("Wavelength (µm)")
# ax.set_ylabel("RSR")
# ax.set_title("Band 12, 13, and 15 RSR (Threshold-Based Limits)")
# ax.legend()

# plt.tight_layout()
# plt.show()

# # Print band limits
# print(f"NIR1 Band Limits: [{band_limits['NIR1'][0]:.4f} µm, {band_limits['NIR1'][1]:.4f} µm]")
# print(f"Water Vapor Band Limits: [{band_limits['Water_Vapor'][0]:.4f} µm, {band_limits['Water_Vapor'][1]:.4f} µm]")
# print(f"SWIR2a Band Limits: [{band_limits['SWIR2a'][0]:.4f} µm, {band_limits['SWIR2a'][1]:.4f} µm]")

# if __name__ == "__main__":
#     pass